In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import skimage.morphology as smo
from skimage.io import imread
import itertools

from sklearn.model_selection import train_test_split

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
os.chdir("D:/GitHub/PhD_Repository/Datasets/MPEG7_CE-Shape-1_Part_B")

Default GPU Device: /device:GPU:0


In [2]:
def image_add_border(image, x , y, val, rel = False):

    """
    Adds a border to the input image
    rel = True - x,y - size of the added border
    rel = False - x,y - size of the image with added border
    val - pixel value 
    """
    
    (sx_org, sy_org) = image.shape
    if rel:
        sx = sx_org + 2*x
        sy = sy_org + 2*y
    else:
        sx = x
        sy = y
    img = np.ones([sx, sy]) * val
    [cx, cy] = ((np.array([sx, sy]) - np.array([sx_org, sy_org]))/2).astype('int')
    img[cx:(cx + sx_org), cy:(cy + sy_org)] = image
    return img

# ---------- #
# Use example:
# obraz = imread("camel-19.gif")
# if len(obraz.shape) == 3:
#     obraz = obraz[:,:,1]
# print(obraz.shape)
# obraz_ = image_add_border(obraz, x = 100, y = 100, val = 1, rel = True)
# print(obraz_.shape)
# plt.subplot(1, 2, 1)
# plt.imshow(obraz)
# plt.subplot(1, 2, 2)
# plt.imshow(obraz_)
# plt.show()
# ---------- #

In [3]:
def get_structuring_elements(n, type = 'disk', initsize = 1, step = 1):
    
    """
    generate a list of n structuring elements
    type = 'disk', 'square', 'diamond'
    """
    
    selist = []
    sesize = initsize
    
    for i in range(n):
        if (type == 'disk'):
            se = smo.disk(sesize)
        if (type == 'square'):
            se = smo.square(2 * sesize + 1)
        if (type == 'diamond'):
            se = smo.diamond(sesize)       
        
        selist.append(se)
        sesize += step
        
    return selist

# ---------- #
# Use examples:
# get_structuring_elements(3, type = "square")
# get_structuring_elements(3, type = "disk")
# get_structuring_elements(3, type = "diamond")
# ---------- #

In [4]:
def morphological_stack(input_image,
                        structuring_elements_depth,
                        transormation_type = 'cv_oc',
                        structuring_elements_type = 'disk',
                        structuring_elements_initsize = 1,
                        structuring_elements_step = 1,
                        addborder = True):
    """
    produce a stack of results of the morphological dual operators
    input_image - imput image (binary or graylevel 2D image)
    structuring_elements_depth - list of two values - numers of up-stack and down-stack images
    transormation_type - type of operations erosion/dilation <-> opening/closing; skimage binary <-> skimage graytone <-> opencv
    structuring_elements_type = structuring element type ('disk', 'square', 'diamond')
    structuring_elements_initsize = initial size of the structuring element 
    structuring_elements_step = increment of the structuring element size
    addborder = True if the external boundary is added, = False otherwise
    """

    max_up = structuring_elements_depth[0] # number of up-stack images (higher indeces, dilation/opening)
    max_down = structuring_elements_depth[1] # number of down-stack images (kower indeces, erosion/closing)
    max_updown = max(max_up, max_down)
    
    structuring_elements_list = get_structuring_elements(n = max_updown,
                                                         type = structuring_elements_type, 
                                                         initsize = structuring_elements_initsize,
                                                         step = structuring_elements_step)
    if addborder:
        image = image_add_border(image = input_image,
                                 x = max_updown,
                                 y = max_updown, 
                                 val = 0, 
                                 rel = True)
    else:
        image = input_image
    
    image_out = np.zeros([image.shape[0], image.shape[1], max_up + max_down + 1])    
    count = 0

    if transormation_type == 'b_ed': # binary erosion/dilation - scikit.image
        opencv = False
        operator_down = smo.binary_erosion
        operator_up = smo.binary_dilation
    elif transormation_type == 'b_oc': # binary opening/closing - scikit.image
        opencv = False
        operator_down = smo.binary_opening
        operator_up = smo.binary_closing      
    elif transormation_type == 'ed': # erosion/dilation - scikit.image     
        opencv = False
        operator_down = smo.erosion
        operator_up = smo.dilation
    elif transormation_type == 'oc': # opening/closing - scikit.image
        opencv = False
        operator_down = smo.opening
        operator_up = smo.closing      
    elif transormation_type == 'cv_ed': # erosion/dilation - openCV 
        opencv = True
        operator_down = cv2.MORPH_ERODE
        operator_up = cv2.MORPH_DILATE
    else: # transormation_type == 'cv_oc': # opening/closing - openCV 
        opencv = True
        operator_down = cv2.MORPH_OPEN
        operator_up = cv2.MORPH_CLOSE     
    
    if opencv:  # opencv version 
        for i in range(max_down):
            image_out[:,:,count] = cv2.morphologyEx(image,
                                                    operator_down,
                                                    structuring_elements_list[max_down - i - 1]); count += 1
        image_out[:,:,count] = image; count += 1
        for i in range(max_up):
            image_out[:,:,count] = cv2.morphologyEx(image,
                                                    operator_up, 
                                                    structuring_elements_list[i]); count += 1
        
    else:   # scikit image version
        for i in range(max_down):
            operator_down(image,
                          selem = structuring_elements_list[max_down - i - 1],
                          out = image_out[:,:,count]); count += 1
        image_out[:,:,count] = image; count += 1
        for i in range(max_up):
            operator_up(image,
                        selem = structuring_elements_list[i], 
                        out = image_out[:,:,count]); count += 1   
        
    return image_out

# ---------- #
# Use examples:
# obraz = imread("camel-19.gif")
# if len(obraz.shape) == 3:
#     obraz = obraz[:,:,1]
# obraz_ = morphological_stack(input_image = obraz,
#                              structuring_elements_depth = [10, 10])
# print(obraz_.shape)
# plt.imshow(obraz_[:,:,20])
# ---------- #

In [5]:
def flat_morphological_stack(input_stack, normalize = True):

    """
    flat morphological stack
    normalize - convert result for pixel values between 0-1
    """
    
    max_up_down = input_stack.shape[2]
    max_value = input_stack.max()
    image_out = np.sum(input_stack/max_value, axis = 2)
    if normalize:
            image_out /= max_up_down
    
    return image_out

# ---------- #
# Use examples:
# obraz = imread("camel-19.gif")
# if len(obraz.shape) == 3:
#     obraz = obraz[:,:,1]
# obraz_ = morphological_stack(input_image = obraz,
#                              structuring_elements_depth = [25, 25],
#                             transormation_type = "cv_ed")
# obraz__ = flat_morphological_stack(input_stack = obraz_)
# plt.imshow(obraz__)

# obraz = imread("camel-19.gif")
# if len(obraz.shape) == 3:
#     obraz = obraz[:,:,1]
# obraz_ = morphological_stack(input_image = obraz,
#                              structuring_elements_depth = [25, 25],
#                             transormation_type = "cv_oc")
# obraz__ = flat_morphological_stack(input_stack = obraz_)
# plt.imshow(obraz__)
# ---------- #

In [6]:
def morphological_stack_with_flat(input_image,
                                  structuring_elements_depth,
                                  transormation_type = 'cv_oc',
                                  structuring_elements_type = 'disk',
                                  structuring_elements_initsize = 1,
                                  structuring_elements_step = 1,
                                  addborder = True,
                                  normalize_flat = True):
    """
    produce a stack of results of the morphological dual operators
    input_image - imput image (binary or graylevel 2D image)
    structuring_elements_depth - list of two values - numers of up-stack and down-stack images
    transormation_type - type of operations erosion/dilation <-> opening/closing; skimage binary <-> skimage graytone <-> opencv
    structuring_elements_type = structuring element type ('disk', 'square', 'diamond')
    structuring_elements_initsize = initial size of the structuring element 
    structuring_elements_step = increment of the structuring element size
    addborder = True if the external boundary is added, = False otherwise
    """

    max_up = structuring_elements_depth[0] # number of up-stack images (higher indeces, dilation/opening)
    max_down = structuring_elements_depth[1] # number of down-stack images (kower indeces, erosion/closing)
    max_updown = max(max_up, max_down)
    
    structuring_elements_list = get_structuring_elements(n = max_updown,
                                                         type = structuring_elements_type, 
                                                         initsize = structuring_elements_initsize,
                                                         step = structuring_elements_step)
    if addborder:
        image = image_add_border(image = input_image,
                                 x = max_updown,
                                 y = max_updown, 
                                 val = 0, 
                                 rel = True)
    else:
        image = input_image
    
    image_out = np.zeros([image.shape[0], image.shape[1], max_up + max_down + 1])    
    count = 0

    if transormation_type == 'b_ed': # binary erosion/dilation - scikit.image
        opencv = False
        operator_down = smo.binary_erosion
        operator_up = smo.binary_dilation
    elif transormation_type == 'b_oc': # binary opening/closing - scikit.image
        opencv = False
        operator_down = smo.binary_opening
        operator_up = smo.binary_closing      
    elif transormation_type == 'ed': # erosion/dilation - scikit.image     
        opencv = False
        operator_down = smo.erosion
        operator_up = smo.dilation
    elif transormation_type == 'oc': # opening/closing - scikit.image
        opencv = False
        operator_down = smo.opening
        operator_up = smo.closing      
    elif transormation_type == 'cv_ed': # erosion/dilation - openCV 
        opencv = True
        operator_down = cv2.MORPH_ERODE
        operator_up = cv2.MORPH_DILATE
    else: # transormation_type == 'cv_oc': # opening/closing - openCV 
        opencv = True
        operator_down = cv2.MORPH_OPEN
        operator_up = cv2.MORPH_CLOSE     
    
    if opencv:  # opencv version 
        for i in range(max_down):
            image_out[:,:,count] = cv2.morphologyEx(image,
                                                    operator_down,
                                                    structuring_elements_list[max_down - i - 1]); count += 1
        image_out[:,:,count] = image; count += 1
        for i in range(max_up):
            image_out[:,:,count] = cv2.morphologyEx(image,
                                                    operator_up, 
                                                    structuring_elements_list[i]); count += 1
        
    else:   # scikit image version
        for i in range(max_down):
            operator_down(image,
                          selem = structuring_elements_list[max_down - i - 1],
                          out = image_out[:,:,count]); count += 1
        image_out[:,:,count] = image; count += 1
        for i in range(max_up):
            operator_up(image,
                        selem = structuring_elements_list[i], 
                        out = image_out[:,:,count]); count += 1   
            
    image_out = flat_morphological_stack(input_stack = image_out, normalize = normalize_flat)
        
    return image_out

# ---------- #
# Use examples:
# obraz = imread("camel-19.gif")
# if len(obraz.shape) == 3:
#     obraz = obraz[:,:,1]
# obraz_ = morphological_stack_with_flat(input_image = obraz,
#                                        transormation_type = "cv_oc",
#                                        structuring_elements_depth = [25, 25])
# obraz_.shape
# plt.imshow(obraz_)

# obraz = imread("camel-19.gif")
# if len(obraz.shape) == 3:
#     obraz = obraz[:,:,1]
# obraz_ = morphological_stack_with_flat(input_image = obraz,
#                                        transormation_type = "cv_ed",
#                                        structuring_elements_depth = [25, 25])
# obraz_.shape
# plt.imshow(obraz_)
# ---------- #

In [7]:
def build_model(image_size, channels, start_neurons, dense_neurons, classes, model_name = "model_1"):
     
    input_tensor = tf.keras.layers.Input(shape = [image_size, image_size, channels])

    conv_1 = tf.keras.layers.Conv2D(filters = start_neurons * 1, 
                                    kernel_size = (3, 3),
                                    strides = (1, 1), 
                                    activation = tf.keras.activations.relu,
                                    padding = "same")(input_tensor)
    conv_2 = tf.keras.layers.Conv2D(filters = start_neurons * 1, 
                                    kernel_size = (3, 3),
                                    strides = (1, 1), 
                                    activation = tf.keras.activations.relu,
                                    padding = "same")(conv_1)
    conv_3 = tf.keras.layers.Conv2D(filters = start_neurons * 1, 
                                    kernel_size = (3, 3),
                                    strides = (1, 1), 
                                    activation = tf.keras.activations.relu,
                                    padding = "same")(conv_2)
    pool_1 = tf.keras.layers.MaxPool2D(pool_size = (2, 2),
                                       strides = (2, 2))(conv_3)

    flatten = tf.keras.layers.Flatten()(pool_1)
    dense = tf.keras.layers.Dense(units = dense_neurons, activation = tf.keras.activations.relu)(flatten)
    output_tensor = tf.keras.layers.Dense(units = classes, activation = tf.keras.activations.softmax)(dense)

    model = tf.keras.models.Model(inputs = input_tensor, 
                                  outputs = output_tensor, 
                                  name = model_name)
    
    return model

In [8]:
def morphological_test_cifar_10(results_directory = "D:/GitHub/PhD_Repository/Results_4/",
                                model_name_ = "model",
                                structuring_elements_depth_ = [2, 2],
                                transormation_type_ = "cv_ed",
                                structuring_elements_type_ = "disk",
                                structuring_elements_initsize_ = 1,
                                structuring_elements_step_ = 1,
                                addborder_ = True,
                                morphological_transformation_mode_ = True,
                                start_neurons_ = 16,
                                dense_neurons_ = 256,
                                epochs_ = 10,
                                batch_size_ = 32,
                                augmentation_ = True):

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    if morphological_transformation_mode_ == True:

        print("1.train. Convert RGB to greyscale")
        x_train_1 = [img[:, :, 0] * 0.3 + img[:, :, 1] * 0.6 + img[:, :, 2] * 0.1 for img in x_train]

        print("2.train. Morphological operations")
        x_train_2 = [morphological_stack_with_flat(input_image = i,
                                                   structuring_elements_depth = structuring_elements_depth_,
                                                   transormation_type = transormation_type_,
                                                   structuring_elements_type = structuring_elements_type_,
                                                   structuring_elements_initsize = structuring_elements_initsize_,
                                                   structuring_elements_step = structuring_elements_step_,
                                                   addborder = addborder_) for i in x_train_1]
        del x_train_1

        print("3.train. Expand first dim")
        x_train_3 = [np.expand_dims(i, 0) for i in x_train_2]
        del x_train_2

        print("4.train. Crop images")
        shp = x_train_3[0].shape[1]
        sed_max = np.max(structuring_elements_depth_)
        x_train_4 = [i[:, sed_max:shp - sed_max, sed_max:shp - sed_max] for i in x_train_3]
        del x_train_3

        print("5.train. Expand first dim")
        x_train_5 = [np.expand_dims(i, -1) for i in x_train_4]
        del x_train_4

        print("6.train. Convert list to numpy array")
        x_train_6 = np.concatenate(np.array(x_train_5), axis = 0)
        del x_train_5

        x_train_ = x_train_6
        y_train_ = y_train

        print("1.test. Convert RGB to greyscale")
        x_test_1 = [img[:, :, 0] * 0.3 + img[:, :, 1] * 0.6 + img[:, :, 2] * 0.1 for img in x_test]

        print("2.test. Morphological operations")
        x_test_2 = [morphological_stack_with_flat(input_image = i,
                                                   structuring_elements_depth = structuring_elements_depth_,
                                                   transormation_type = transormation_type_,
                                                   structuring_elements_type = structuring_elements_type_,
                                                   structuring_elements_initsize = structuring_elements_initsize_,
                                                   structuring_elements_step = structuring_elements_step_,
                                                   addborder = addborder_) for i in x_test_1]
        del x_test_1

        print("3.test. Expand first dim")
        x_test_3 = [np.expand_dims(i, 0) for i in x_test_2]
        del x_test_2

        print("4.test. Crop images")
        shp = x_test_3[0].shape[1]
        sed_max = np.max(structuring_elements_depth_)
        x_test_4 = [i[:, sed_max:shp - sed_max, sed_max:shp - sed_max] for i in x_test_3]
        del x_test_3

        print("5.test. Expand first dim")
        x_test_5 = [np.expand_dims(i, -1) for i in x_test_4]
        del x_test_4

        print("6.test. Convert list to numpy array")
        x_test_6 = np.concatenate(np.array(x_test_5), axis = 0)
        del x_test_5

        x_test_ = x_test_6
        y_test_ = y_test

    else:

        print("1.train. Normal")
        x_train_ = x_train
        y_train_ = y_train

        print("1.test. Normal")
        x_test_ = x_test
        y_test_ = y_test

        print("Split: train, validation and test")

    np.random.seed(42)
    tf.random.set_seed(42)
    X = np.arange(x_train_.shape[0])
    y = np.arange(x_train_.shape[0])

    classes = len(np.unique(y_train_))

    x_train_id, x_validation_id, y_train_id, y_validation_id = train_test_split(X, y, test_size = 0.2, random_state = 42)

    x_train__ = x_train_[x_train_id, :, :, :]
    y_train__ = y_train_[y_train_id]

    x_valid__ = x_train_[x_validation_id, :, :, :]
    y_valid__ = y_train_[y_validation_id]

    x_test__ = x_test_
    y_test__ = y_test_

    y_train__ = tf.keras.utils.to_categorical(y_train__, classes)
    y_valid__ = tf.keras.utils.to_categorical(y_valid__, classes)
    y_test__ = tf.keras.utils.to_categorical(y_test__, classes)

    print(x_train__.shape)
    print(y_train__.shape)
    print(x_valid__.shape)
    print(y_valid__.shape)
    print(x_test__.shape)
    print(y_test__.shape)

    image_size = x_train__.shape[1]
    channels = x_train__.shape[3]
    early_stopping = int(epochs_ * 0.1)

    print("Build data generators")
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 90,
            width_shift_range = np.ceil(0.1 * image_size), 
            height_shift_range = np.ceil(0.1 * image_size),
            horizontal_flip = True, 
            vertical_flip = True,
            fill_mode = 'nearest')

    test_validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 0,
            width_shift_range = 0, 
            height_shift_range = 0,
            horizontal_flip = False, 
            vertical_flip = False,
            fill_mode = 'nearest')

    callbacks = [tf.keras.callbacks.EarlyStopping(patience = early_stopping, monitor = 'val_accuracy', verbose = 1),
                 tf.keras.callbacks.ModelCheckpoint(filepath = "D:/GitHub/PhD_Repository/Results_4/weights.h5",
                                                    save_weights_only=True,
                                                    monitor='val_accuracy',
                                                    mode='max',
                                                    save_best_only=True)]

    model = build_model(image_size = image_size,
                        channels = channels, 
                        start_neurons = start_neurons_,
                        dense_neurons = dense_neurons_, 
                        classes = classes,
                        model_name = model_name_)
    # print(model.summary())

    model.compile(optimizer = tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.categorical_crossentropy,
                      metrics = ["accuracy"])

    tf.keras.backend.clear_session()

    if augmentation_ == True:
        print("Augmentation mode on")
        train_generator = train_datagen.flow(x_train__, y_train__, batch_size = batch_size_)
    else:
        print("Augmentation mode off")
        train_generator = test_validation_datagen.flow(x_train__, y_train__, batch_size = batch_size_)

    validation_generator = test_validation_datagen.flow(x_valid__, y_valid__, batch_size = batch_size_)

    model_results = model.fit(train_generator,
                              validation_data = validation_generator,
                              steps_per_epoch = np.ceil(train_generator.n / batch_size_),
                              validation_steps = np.ceil(validation_generator.n / batch_size_),
                              epochs = epochs_,
                              shuffle = True,
                              callbacks = callbacks)

    model = build_model(image_size = image_size,
                        channels = channels, 
                        start_neurons = start_neurons_,
                        dense_neurons = dense_neurons_, 
                        classes = classes,
                        model_name = model_name_)

    model.load_weights("D:/GitHub/PhD_Repository/Results_4/weights.h5")

    model.compile(optimizer = tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.categorical_crossentropy,
                      metrics = ["accuracy"])

    model_results_pd = pd.DataFrame(model_results.history)
    model_results_pd["model"] = model_name_
    model_results_pd["epoch"] = np.array(model_results.epoch) + 1
    model_results_pd["structuring_elements_depth"] = str(structuring_elements_depth_)
    model_results_pd["transormation_type"] = transormation_type_
    model_results_pd["structuring_elements_type"] = structuring_elements_type_
    model_results_pd["structuring_elements_initsize"] = structuring_elements_initsize_
    model_results_pd["structuring_elements_step"] = structuring_elements_step_
    model_results_pd["morphological_transformation_mode"] = morphological_transformation_mode_
    model_results_pd["start_neurons"] = start_neurons_
    model_results_pd["dense_neurons"] = dense_neurons_
    model_results_pd["epochs"] = epochs_
    model_results_pd["batch_size"] = batch_size_
    model_results_pd["augmentation"] = augmentation_
    model_results_pd.to_csv(results_directory + model_name_ + "_history.csv")

    model_train_accuracy = model.evaluate(x_train__, y_train__)[1]
    model_validation_accuracy = model.evaluate(x_valid__, y_valid__)[1]
    model_test_accuracy = model.evaluate(x_test__, y_test__)[1]

    print("Train accuracy:", model_train_accuracy)
    print("Validation accuracy:", model_validation_accuracy)
    print("Test accuracy:", model_test_accuracy)

    evaluation_results = pd.DataFrame({"Dataset" : ["train", "validation", "test"],
                                       "Accuracy" : [model_train_accuracy, model_validation_accuracy, model_test_accuracy],
                                       "Model_Name" : [model_name_] * 3})
    evaluation_results["epochs"] = model_results_pd.shape[0]
    evaluation_results["structuring_elements_depth"] = str(structuring_elements_depth_)
    evaluation_results["transormation_type"] = transormation_type_
    evaluation_results["structuring_elements_type"] = structuring_elements_type_
    evaluation_results["structuring_elements_initsize"] = structuring_elements_initsize_
    evaluation_results["structuring_elements_step"] = structuring_elements_step_
    evaluation_results["morphological_transformation_mode"] = morphological_transformation_mode_
    evaluation_results["start_neurons"] = start_neurons_
    evaluation_results["dense_neurons"] = dense_neurons_
    evaluation_results["epochs"] = epochs_
    evaluation_results["batch_size"] = batch_size_
    evaluation_results["augmentation"] = augmentation_
    evaluation_results.to_csv(results_directory + model_name_ + "_evaluation.csv")

    del model
    del model_results_pd
    del evaluation_results
    os.remove("D:/GitHub/PhD_Repository/Results_4/weights.h5")

In [9]:
def expandgrid(*itrs):
    product = list(itertools.product(*itrs))
    return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

In [10]:
structuring_elements_depth_hp = [[1, 1], [2, 2], [3, 3], [4, 4]]
transormation_type_hp = ["cv_ed", "cv_oc"]
structuring_elements_type_hp = ["disk", "square", "diamond"]
structuring_elements_step_hp = [1, 2]
start_neurons_hp = [16, 32]
dense_neurons_hp = [128, 256]

grid_big = pd.DataFrame(expandgrid(structuring_elements_depth_hp,
                               transormation_type_hp,
                               structuring_elements_type_hp,
                               structuring_elements_step_hp,
                               start_neurons_hp,
                               dense_neurons_hp))
grid_big["id"] = list(np.arange(grid_big.shape[0]))
grid_big.columns = ["structuring_elements_depth_hp", 
                "transormation_type_hp", 
                "structuring_elements_type_hp",
                "structuring_elements_step_hp", 
                "start_neurons_hp", 
                "dense_neurons_hp",
                "id"]
grid_big

,structuring_elements_depth_hp,transormation_type_hp,structuring_elements_type_hp,structuring_elements_step_hp,start_neurons_hp,dense_neurons_hp,id
0,"[1, 1]",cv_ed,disk,1,16,128,0
1,"[1, 1]",cv_ed,disk,1,16,256,1
2,"[1, 1]",cv_ed,disk,1,32,128,2
3,"[1, 1]",cv_ed,disk,1,32,256,3
4,"[1, 1]",cv_ed,disk,2,16,128,4
5,"[1, 1]",cv_ed,disk,2,16,256,5
6,"[1, 1]",cv_ed,disk,2,32,128,6
7,"[1, 1]",cv_ed,disk,2,32,256,7
8,"[1, 1]",cv_ed,square,1,16,128,8
9,"[1, 1]",cv_ed,square,1,16,256,9


In [11]:
start_neurons_hp = [16, 32]
dense_neurons_hp = [128, 256]

grid_small = pd.DataFrame(expandgrid(start_neurons_hp,
                                     dense_neurons_hp))
grid_small["id"] = list(np.arange(grid_small.shape[0]))
grid_small.columns = ["start_neurons_hp", 
                "dense_neurons_hp",
                "id"]
grid_small.head()

,start_neurons_hp,dense_neurons_hp,id
0,16,128,0
1,16,256,1
2,32,128,2
3,32,256,3


In [12]:
for i in np.arange(grid_small.shape[0]):
    
    model_name_ = "small_grid_model_" + str(i)
    print(model_name_)
    
    morphological_test_cifar_10(results_directory = "D:/GitHub/PhD_Repository/Results_4/",
                                model_name_ = model_name_,
                                structuring_elements_depth_ = [0, 0],
                                transormation_type_ = None,
                                structuring_elements_type_ = None,
                                structuring_elements_initsize_ = None,
                                structuring_elements_step_ = None,
                                addborder_ = None,
                                morphological_transformation_mode_ = False,
                                start_neurons_ = grid_small["start_neurons_hp"][i],
                                dense_neurons_ = grid_small["dense_neurons_hp"][i],
                                epochs_ = 100,
                                batch_size_ = 512,
                                augmentation_ = True)

small_grid_model_0
1.train. Normal
1.test. Normal
Split: train, validation and test
(40000, 32, 32, 3)
(40000, 10)
(10000, 32, 32, 3)
(10000, 10)
(10000, 32, 32, 3)
(10000, 10)
Build data generators
Augmentation mode on
Epoch 1/100
79/79 [==============================] - 11s 145ms/step - loss: 5.2615 - accuracy: 0.1555 - val_loss: 2.1639 - val_accuracy: 0.2101
Epoch 2/100
79/79 [==============================] - 11s 143ms/step - loss: 2.0684 - accuracy: 0.2371 - val_loss: 2.0333 - val_accuracy: 0.2620
Epoch 3/100
79/79 [==============================] - 11s 142ms/step - loss: 1.9863 - accuracy: 0.2720 - val_loss: 2.0324 - val_accuracy: 0.2798
Epoch 4/100
79/79 [==============================] - 11s 144ms/step - loss: 1.9449 - accuracy: 0.2945 - val_loss: 1.8977 - val_accuracy: 0.3174
Epoch 5/100
79/79 [==============================] - 11s 142ms/step - loss: 1.8964 - accuracy: 0.3113 - val_loss: 1.9112 - val_accuracy: 0.3214
Epoch 6/100
79/79 [==============================] - 11s 144

In [14]:
for i in np.arange(grid_big.shape[0]):
    
    model_name_ = "big_grid_model_" + str(i)
    print(model_name_)
    
    morphological_test_cifar_10(results_directory = "D:/GitHub/PhD_Repository/Results_4/",
                                model_name_ = model_name_,
                                structuring_elements_depth_ = grid_big["structuring_elements_depth_hp"][i],
                                transormation_type_ = grid_big["transormation_type_hp"][i],
                                structuring_elements_type_ = grid_big["structuring_elements_type_hp"][i],
                                structuring_elements_initsize_ = 1,
                                structuring_elements_step_ = grid_big["structuring_elements_step_hp"][i],
                                addborder_ = True,
                                morphological_transformation_mode_ = True,
                                start_neurons_ = grid_big["start_neurons_hp"][i],
                                dense_neurons_ = grid_big["dense_neurons_hp"][i],
                                epochs_ = 100,
                                batch_size_ = 512,
                                augmentation_ = True)

big_grid_model_0
1.train. Convert RGB to greyscale
2.train. Morphological operations
3.train. Expand first dim
4.train. Crop images
5.train. Expand first dim
6.train. Convert list to numpy array
1.test. Convert RGB to greyscale
2.test. Morphological operations
3.test. Expand first dim
4.test. Crop images
5.test. Expand first dim
6.test. Convert list to numpy array
(40000, 32, 32, 1)
(40000, 10)
(10000, 32, 32, 1)
(10000, 10)
(10000, 32, 32, 1)
(10000, 10)
Build data generators
Augmentation mode on
Epoch 1/100
79/79 [==============================] - 6s 78ms/step - loss: 2.2233 - accuracy: 0.1761 - val_loss: 2.1416 - val_accuracy: 0.2165
Epoch 2/100
79/79 [==============================] - 6s 78ms/step - loss: 2.1223 - accuracy: 0.2193 - val_loss: 2.0559 - val_accuracy: 0.2610
Epoch 3/100
79/79 [==============================] - 6s 78ms/step - loss: 2.0759 - accuracy: 0.2436 - val_loss: 2.0188 - val_accuracy: 0.2696
Epoch 4/100
79/79 [==============================] - 6s 78ms/step - los